In [1]:
import datetime

import matplotlib.pyplot as plt

from acsql.database.database_interface import session
from acsql.database.database_interface import WFC_raw_0
from acsql.database.database_interface import HRC_raw_0
from acsql.database.database_interface import SBC_raw_0

In [2]:
def get_dates(detector):
    """Return a list of observation dates and the aggregate number of
    observations for every observation for the given detector over time.
    
    Parameters
    ----------
    detector : obj
        The SQLAlchemy Table object for the detector.
        
    Returns
    -------
    dates : list
        A list of observation dates
    num_obs : list
        A list of the aggregate number of observations over time for each
        date in the dates list.
    """
    
    results = session.query(getattr(detector,'date-obs')).all()
    dates = sorted([item[0] for item in results])
    results_dict = {}
    num_obs = 1
    for date in dates:
        results_dict[date] = num_obs
        num_obs += 1
    dates, num_obs = zip(*sorted(results_dict.items()))
    dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

    return dates, num_obs

In [3]:
wfc_dates, wfc_num_obs = get_dates(WFC_raw_0)
hrc_dates, hrc_num_obs  = get_dates(HRC_raw_0)
sbc_dates, sbc_num_obs  = get_dates(SBC_raw_0)

In [16]:
plt.rcParams['font.size'] = 14
plt.style.use('bmh')
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.set_ylabel('Number of Observations')
ax.set_yticks([0, 20000, 40000, 60000, 80000, 100000, 120000])
ax.set_yticklabels(['0', '20k', '40k', '60k', '80k', '100k', '120k'])
ax.plot(wfc_dates, wfc_num_obs, linewidth=3, color='#348ABD', label='WFC')
ax.plot(hrc_dates, hrc_num_obs, linewidth=3, color='#A60628', label='HRC')
ax.plot(sbc_dates, sbc_num_obs, linewidth=3, color='#467821', label='SBC')
plt.legend()
plt.tight_layout()
plt.savefig('figures/num_obs.png')

In [5]:
wfc_num_obs

(20,
 61,
 118,
 124,
 129,
 134,
 139,
 144,
 162,
 167,
 180,
 185,
 190,
 203,
 211,
 229,
 237,
 262,
 306,
 339,
 353,
 358,
 383,
 404,
 425,
 471,
 505,
 555,
 609,
 640,
 659,
 668,
 731,
 778,
 838,
 895,
 919,
 1065,
 1097,
 1123,
 1139,
 1225,
 1246,
 1261,
 1266,
 1287,
 1303,
 1318,
 1350,
 1379,
 1400,
 1444,
 1479,
 1513,
 1525,
 1564,
 1601,
 1616,
 1631,
 1644,
 1657,
 1666,
 1685,
 1716,
 1731,
 1748,
 1771,
 1798,
 1815,
 1868,
 1877,
 1914,
 1937,
 1958,
 1998,
 2022,
 2034,
 2046,
 2075,
 2093,
 2115,
 2147,
 2167,
 2191,
 2203,
 2221,
 2246,
 2270,
 2284,
 2310,
 2334,
 2366,
 2486,
 2507,
 2522,
 2540,
 2550,
 2555,
 2581,
 2604,
 2614,
 2631,
 2643,
 2666,
 2697,
 2756,
 2796,
 2828,
 2851,
 2874,
 2898,
 2927,
 2940,
 2955,
 2980,
 3010,
 3032,
 3053,
 3205,
 3227,
 3246,
 3258,
 3271,
 3285,
 3328,
 3344,
 3367,
 3382,
 3412,
 3435,
 3467,
 3510,
 3547,
 3586,
 3602,
 3610,
 3626,
 3659,
 3721,
 3779,
 3822,
 3856,
 3902,
 3912,
 3938,
 3966,
 4013,
 4039,
 40